# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\lucas\OneDrive\Documentos\Cdados\Projeto 1\22-2a-cd-p1-grupo_rodrigol5\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [4]:
filename = '../data/dados.xlsx'

In [5]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Comprar ou vender?,"Com commodities caindo, correr para as small c...",A piora do cenário macro global levou a uma qu...,02/08/2022 - 17:52,36,0
1,Bula do Mercado,Ibovespa hoje: Marasmo no exterior e agenda va...,O Ibovespa pode receber influência do sinal ne...,19/08/2022 - 8:07,22,0
2,Fusões & Aquisições,Prisma Capital se torna titular de 56% da Domm...,"A Dommo Energia (DMMO3), antiga OGX de Eike Ba...",06/09/2022 - 8:07,4,0
3,Combustíveis,Bolsonaro diz que diesel importado da Rússia p...,"O presidente Jair Bolsonaro afirmou, nesta seg...",11/07/2022 - 11:45,67,1
4,Mercados,Petrobras (PETR4) salta 3% com reinvestimento ...,A Petrobras (PETR3; PETR4) fechou o último pre...,31/08/2022 - 17:25,9,2


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,MT Labs,Não é Petrobras: esta petrolífera pode valoriz...,"Ao abrir um site de notícias sobre economia, é...",18/07/2022 - 12:00,56,2
1,Economia,"Preço do diesel sobe mais de 60%, aponta IPCA-15","No acumulado dos últimos 12 meses, o preço do ...",26/07/2022 - 10:34,46,2
2,Zona do Euro,Congelamento de preços na França prova que a E...,"Ontem, o Carrefour anunciou que vai congelar o...",23/08/2022 - 16:28,19,0
3,Europa,Índice europeu STOXX 600 recua pressionado por...,O índice STOXX 600 fechou em baixa nesta quart...,07/09/2022 - 14:16,2,0
4,Moedas,"Nervosismo externo faz dólar subir para R$ 5,37",A trégua no mercado financeiro durou pouco. Ap...,11/07/2022 - 19:35,66,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

0 - irrelevante,
<br>1 - relevante,
<br>2 - muito relevante.

É classificado a partir desses targets, se mereçem destaque, ou possam ajudar investidores em petróleo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
import re 


def cleanup(text):

    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [8]:
todas_palavras = []
palavras0 = []
palavras1 = []
palavras2 = []

i = 0
while i < len(train):
    linha = cleanup(str(train['PrimeiroParag'][i]).lower())
    linha_split = linha.split()
    x = train.iloc[i]['Target']
    
    for palavra in linha_split:
        todas_palavras.append(palavra)
        if x == 0:
            palavras0.append(palavra)
        if x == 1:
            palavras1.append(palavra)
        if x == 2:
            palavras2.append(palavra)

    i+=1

In [9]:
tabela_todas_palavras = pd.Series(todas_palavras)
tabela_todas_palavras_absoluta = tabela_todas_palavras.value_counts()
qnt_total = len(tabela_todas_palavras_absoluta)

tabela_palavras0 = pd.Series(palavras0)
tabela_palavras_absoluta0 = tabela_palavras0.value_counts()
qnt0 = len(palavras0)
P_0 = qnt0/qnt_total

tabela_palavras1 = pd.Series(palavras1)
tabela_palavras_absoluta1 = tabela_palavras1.value_counts()
qnt1 = len(palavras1)
P_1 = qnt1/qnt_total

tabela_palavras2 = pd.Series(palavras2)
tabela_palavras_absoluta2 = tabela_palavras2.value_counts()
qnt2 = len(palavras2)
P_2 = qnt2/qnt_total

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [51]:
lista_prob = []
lista_target = []

i = 0
while i < len(test):
    linha = cleanup(str(test['PrimeiroParag'][i]).lower())
    linha_split = linha.split()
    x = train.iloc[i]['Target']
    P_palavra_dado_0 = 1
    P_palavra_dado_1 = 1
    P_palavra_dado_2 = 1

    for palavra in linha_split:
        if palavra in tabela_palavras_absoluta0:
            P_palavra_dado_0 *= (1 + tabela_palavras_absoluta0[palavra])/(qnt0 + qnt_total)
        else:
            P_palavra_dado_0 = 1/qnt0 + qnt_total
        
        if palavra in tabela_palavras_absoluta1:
            P_palavra_dado_1 *= (1 + tabela_palavras_absoluta1[palavra])/(qnt1 + qnt_total)
        else:
            P_palavra_dado_1 = 1/qnt1 + qnt_total
        
        if palavra in tabela_palavras_absoluta2:
            P_palavra_dado_2 *= (1 + tabela_palavras_absoluta2[palavra])/(qnt2 + qnt_total)
        else:
            P_palavra_dado_2 = 1/qnt2 + qnt_total

        P_0_dado_palavra = P_palavra_dado_0 * P_0
        P_1_dado_palavra = P_palavra_dado_1 * P_1
        P_2_dado_palavra = P_palavra_dado_2 * P_2

        P_0_dado_frase = 1
        P_1_dado_frase = 1
        P_2_dado_frase = 1


        if P_0_dado_palavra >= P_1_dado_palavra and P_0_dado_palavra >= P_2_dado_palavra:
            P_0_dado_frase *= P_0_dado_palavra

        if P_1_dado_palavra >= P_0_dado_palavra and P_1_dado_palavra >= P_2_dado_palavra:
            P_1_dado_frase *= P_1_dado_palavra

        if P_2_dado_palavra >= P_1_dado_palavra and P_2_dado_palavra >= P_0_dado_palavra:
            P_2_dado_frase *= P_2_dado_palavra


    #if P_0_dado_frase == P_1_dado_frase:
    #    lista_target.append(0)
    #if P_1_dado_frase == P_2_dado_frase:
        #print('boiafe')
      #  lista_target.append(1)
    #if P_0_dado_frase == P_1_dado_frase:
      #  lista_target.append(0)

    if P_0_dado_frase >= P_1_dado_frase and P_0_dado_frase >= P_2_dado_frase:
        lista_target.append(0)
        lista_prob.append(P_0_dado_frase)

    if P_1_dado_frase >= P_0_dado_frase and P_1_dado_frase >= P_2_dado_frase:
        lista_target.append(1)
        lista_prob.append(P_1_dado_frase)

    if P_2_dado_frase >= P_1_dado_frase and P_2_dado_frase >= P_0_dado_frase:
        lista_target.append(2)
        lista_prob.append(P_2_dado_frase)

    i+=1


In [52]:
print(len(lista_prob))
print(len(lista_target))

418
418


In [ ]:
acertos = 0
erros = 0
for x in range(len(train['Target'])):
    if train['Target'][x] == lista_chute[x]:
        acertos += 1
    if train['Target'][x] != lista_chute[x]:
        erros += 1

def calcular_probabilidade(y, z):
    n_acertos = y
    n_erros = z

    probabilidade_acertos = n_acertos/len(train['Target'])
    probabilidade_erros = n_erros/len(train['Target'])

    lista_resultados = [probabilidade_acertos, probabilidade_erros]

    return lista_resultados

calcular_probabilidade(acertos, erros)

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**